#### Importing the classes and functions required for this & initializing the random number generator to a constant value to ensure we can easily reproduce the results.

In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

numpy.random.seed(7)

In [3]:
# we are going to take top 4000 words of the imdb review
top_words = 4000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

#### there is a high chance that we may have unequal length of the sentence- so MAKE IT EQUAL SIZED VECTOR by padding and truncation


In [4]:
max_review_length = 400
X_train= sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test= sequence.pad_sequences(X_test, maxlen=max_review_length)

In [5]:
print(X_test[25])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    1  146   24
  252  138   14   22    9    2   38  364   23  899   54   45  427  285
   25  100  126  181   11    6  189   22   14    9    4    2    7  112
    6  189   22   13 1133  546    8   30    6  194  189  337    5   13
  215  135   15   14  313 1545    4    2  313    7    2    9    4   65
    7    6  604    7 1170  362   23   68   96    8    6 2319  500  937
 1197    8 1250   46   18    4  314    5   28    6  521   11   19    6
  719    2  725    2    4  375 1972   36   97    6 2943 3808    5 1197
    8 

#### Now we will define, compile and fit our LSTM model
---------
#### embedding layer is the first layer that uses vector of size 30 to represent each words
#### ---------
#### Next is LSTM layer which is containing 100 NEURONS.
#### This is a classification problem so lastly we will use dense output layer with a single neuron and a sigmoid activation function to make 0 & 1 as the prediction for good or bad review.
#### its a binary classification so we will use log loss as our COST FUNCTION (binary_crossentropy in Keras).
#### till here we defined the requirements/parameters of our model now we need to reach global minima for the cost function equation. we can use any optimization technique for our convergence such as Adagrad, RMSProp etc (here we will ADAM optimizer).
#### after deciding the optimization technique we dont want to feed whole data at once so we will use a BATCH SIZE.

In [6]:
embedding_vector_length = 30
model = Sequential()
model.add(Embedding(top_words,embedding_vector_length, input_length = max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
# compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# fitting model
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=2, batch_size=60)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 30)           120000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               52400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 172,501
Trainable params: 172,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
417/417 [==============================] - 46s 32ms/step - loss: 0.5677 - accuracy: 0.6869 - val_loss: 0.3499 - val_accuracy: 0.8551
Epoch 2/2
417/417 [==============================] - 13s 30ms/step - loss: 0.3040 - accuracy: 0.8779 - val_loss: 0.3353 - val_accuracy: 0.8578


In [8]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))

Accuracy: 85.78%
